<a href="https://colab.research.google.com/github/fahimalamabir/datatuneanalytics/blob/main/Your_First_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'geospatial-learn-course-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F348259%2F695175%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T105813Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1e4503babb16b3e5bd947eb6747dc1dd701372bffe24b6144a9c319eb240c33aa4e83cd7fa435ca45684d0882a8a90be170c35b94ee7d660c112aebddecfc7925daef7b22fb641f05a2968e18f83272ee806aece7d3f851074559d55b5c93a09ebc73c095dc7cf3dbbdd1981a2059f44d1b416ace39b4b59d1479d047e275d6706c948e7f3843a022217a09ce0b155c4944bf9609ba099c9573cd7d223b04349da9acca502a18b872b812cc6bf73a9c3edfa82fd3e4e9812b2e0bc9eb10a96f27dff7b17d46adb208f5a5427cb2fef2e8dce713457b13b8710642a832330d1ff31eeab0eee9047d3a81c4809b1c8b22ee0119b5c0e704c5521670e1335f58636,data-for-datavis:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F116573%2F3551030%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T105813Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D323730b4f70b278d67b4fee0050ec3b40e6520ae5b6bf60847f915df91ffbb42da91823043ceaf3394599fef8bc9822ac078864a6e2903eaf4eba29d041cbf121b16321bd728ded8d676e5125f0ffb7b8f4bcbba3ba82002076ff33864fece07656f1f001624fd792aca80154cfb4185ea6723999b706edbe171288a4672c275b7cd4a6757afc33c340621f7354603adab24d06a8582423be4c571cf683d0cba29d0fdf80049522fd20e938fac6bb61e90baff5893fb83d9b790a07da43c45382be2f51bd41c434076233fe5e11410607a3886fa4dd5a45c90ce3d67cde427774eb3c81750e0395bc5edfb3e173e575254ec57b5d047c582026070435cd977a0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Introduction

In this micro-course, you'll learn about different methods to wrangle and visualize **geospatial data**, or data with a geographic location.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/v6ZUGgI.png"><br/>
</center>

Along the way, you'll offer solutions to several real-world problems like:
- Where should a global non-profit expand its reach in remote areas of the Philippines?
- How do purple martins, a threatened bird species, travel between North and South America?  Are the birds travelling to conservation areas?
- Which areas of Japan could potentially benefit from extra earthquake reinforcement?
- Which Starbucks stores in California are strong candidates for the next [Starbucks Reserve Roastery](https://www.forbes.com/sites/garystern/2019/01/22/starbucks-reserve-roastery-its-spacious-and-trendy-but-why-is-starbucks-slowing-down-expansion/#6cb80d4a1bc6) location?
- Does New York City have sufficient hospitals to respond to motor vehicle collisions?  Which areas of the city have gaps in coverage?

You'll also visualize crime in the city of Boston, examine health facilities in Ghana, explore top universities in Europe, and track releases of toxic chemicals in the United States.

In this first tutorial, we'll quickly cover the pre-requisites that you'll need to complete this micro-course.  And, if you'd like to review more deeply, we recommend the **[Pandas micro-course](https://www.kaggle.com/learn/pandas)**.  

We'll also get started with visualizing our first geospatial dataset!

# Reading data

The first step is to read in some geospatial data!  To do this, we'll use the [GeoPandas](http://geopandas.org/) library.

In [ ]:
import geopandas as gpd

There are many, many different geospatial file formats, such as [shapefile](https://en.wikipedia.org/wiki/Shapefile), [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language), and [GPKG](https://en.wikipedia.org/wiki/GeoPackage).  We won't discuss their differences in this micro-course, but it's important to mention that:
- shapefile is the most common file type that you'll encounter, and
- all of these file types can be quickly loaded with the `gpd.read_file()` function.

The next code cell loads a shapefile containing information about forests, wilderness areas, and other lands under the care of the [Department of Environmental Conservation](https://www.dec.ny.gov/index.html) in the state of New York.  

In [ ]:
# Read in the data
full_data = gpd.read_file("../input/geospatial-learn-course-data/DEC_lands/DEC_lands/DEC_lands.shp")

# View the first five rows of the data
full_data.head()

As you can see in the "CLASS" column, each of the first five rows corresponds to a different forest.  

For the rest of this tutorial, consider a scenario where you'd like to use this data to plan a weekend camping trip.  Instead of relying on crowd-sourced reviews online, you decide to create your own map.  This way, you can tailor the trip to your specific interests.

# Prerequisites

To view the first five rows of the data, we used the `head()` method.  You may recall that this is also what we use to preview a Pandas DataFrame.  In fact, every command that you can use with a DataFrame will work with the data!  

This is because the data was loaded into a (GeoPandas) **GeoDataFrame** object that has all of the capabilities of a (Pandas) DataFrame.

In [ ]:
type(full_data)

For instance, if we don't plan to use all of the columns, we can select a subset of them.  (_To review other methods for selecting data, check out [this tutorial](https://www.kaggle.com/residentmario/indexing-selecting-assigning/) from the Pandas micro-course_.)

In [ ]:
data = full_data.loc[:, ["CLASS", "COUNTY", "geometry"]].copy()

We use the `value_counts()` method to see a list of different land types, along with how many times they appear in the dataset. (_To review this (and related methods), check out [this tutorial](https://www.kaggle.com/residentmario/summary-functions-and-maps) from the Pandas micro-course._)

In [ ]:
# How many lands of each type are there?
data.CLASS.value_counts()

You can also use `loc` (and `iloc`) and `isin` to select subsets of the data.  (_To review this, check out [this tutorial](https://www.kaggle.com/residentmario/indexing-selecting-assigning/) from the Pandas micro-course._)

In [ ]:
# Select lands that fall under the "WILD FOREST" or "WILDERNESS" category
wild_lands = data.loc[data.CLASS.isin(['WILD FOREST', 'WILDERNESS'])].copy()
wild_lands.head()

If you're not familiar with the commands above, you are encouraged to bookmark this page for reference, so you can look up the commands as needed.  (_Alternatively, you can take the [Pandas micro-course](https://www.kaggle.com/learn/pandas)._)  We'll use these commands throughout this micro-course to understand and filter data before creating maps.

# Create your first map!

We can quickly visualize the data with the `plot()` method.

In [ ]:
wild_lands.plot()

Every GeoDataFrame contains a special "geometry" column.  It contains all of the geometric objects that are displayed when we call the `plot()` method.

In [ ]:
# View the first five entries in the "geometry" column
wild_lands.geometry.head()

While this column can contain a variety of different datatypes, each entry will typically be a **Point**, **LineString**, or **Polygon**.

![](https://storage.googleapis.com/kaggle-media/learn/images/N1llefr.png)

The "geometry" column in our dataset contains 2983 different Polygon objects, each corresponding to a different shape in the plot above.

In the code cell below, we create three more GeoDataFrames, containing campsite locations (**Point**), foot trails (**LineString**), and county boundaries (**Polygon**).

In [ ]:
# Campsites in New York state (Point)
POI_data = gpd.read_file("../input/geospatial-learn-course-data/DEC_pointsinterest/DEC_pointsinterest/Decptsofinterest.shp")
campsites = POI_data.loc[POI_data.ASSET=='PRIMITIVE CAMPSITE'].copy()

# Foot trails in New York state (LineString)
roads_trails = gpd.read_file("../input/geospatial-learn-course-data/DEC_roadstrails/DEC_roadstrails/Decroadstrails.shp")
trails = roads_trails.loc[roads_trails.ASSET=='FOOT TRAIL'].copy()

# County boundaries in New York state (Polygon)
counties = gpd.read_file("../input/geospatial-learn-course-data/NY_county_boundaries/NY_county_boundaries/NY_county_boundaries.shp")

Next, we create a map from all four GeoDataFrames.  

The `plot()` method takes as (optional) input several parameters that can be used to customize the appearance.  Most importantly, setting a value for `ax` ensures that all of the information is plotted on the same map.

In [ ]:
# Define a base map with county boundaries
ax = counties.plot(figsize=(10,10), color='none', edgecolor='gainsboro', zorder=3)

# Add wild lands, campsites, and foot trails to the base map
wild_lands.plot(color='lightgreen', ax=ax)
campsites.plot(color='maroon', markersize=2, ax=ax)
trails.plot(color='black', markersize=1, ax=ax)

It looks like the northeastern part of the state would be a great option for a camping trip!

# Your turn

This feels complex at first, but you've already learned enough to do important analysis. See for yourself as you **[identify remote areas](https://www.kaggle.com/kernels/fork/5832167)** of the Philippines where a non-profit can expand its operations.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*